## 1. Cài đặt thư viện

In [1]:
!pip install -q llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-vector-stores-faiss
!pip install -q faiss-cpu beautifulsoup4 requests tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, w

## 2. Import libraries và setup

In [2]:
import os
import json
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from typing import List, Dict

# LlamaIndex
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# Setup OpenAI API key từ Colab Secrets
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

print("✅ Setup completed!")

✅ Setup completed!


## 3. Cấu hình LlamaIndex

In [3]:
# Cấu hình LLM và Embedding
llm = OpenAI(model="gpt-4o-mini", temperature=0.7)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Set global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 50

print("✅ LlamaIndex configured with GPT-4o-mini")

✅ LlamaIndex configured with GPT-4o-mini


## 4. Crawl văn bản pháp luật từ VBPL

In [7]:
def crawl_legal_document(url: str) -> str:
    """
    Crawl nội dung văn bản pháp luật từ VBPL.vn
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm nội dung chính - VBPL thường dùng các selectors này
        # Thử nhiều cách để tìm nội dung
        content_text = ""

        # Cách 1: Tìm div có id hoặc class chứa "content", "fulltext", "noidung"
        selectors = [
            {'id': 'divContent'},
            {'id': 'ctl00_Content'},
            {'class': 'content1'},
            {'class': 'fulltext'},
            {'class': 'noidung'},
            {'id': 'content'},
        ]

        for selector in selectors:
            content_div = soup.find('div', selector)
            if content_div:
                content_text = content_div.get_text(separator='\n', strip=True)
                if len(content_text) > 500:  # Nội dung đủ dài
                    break

        # Cách 2: Nếu không tìm thấy, tìm trong body và loại bỏ script, style
        if len(content_text) < 500:
            # Xóa các thẻ không cần thiết
            for tag in soup(['script', 'style', 'nav', 'header', 'footer', 'aside']):
                tag.decompose()

            # Lấy text từ body
            body = soup.find('body')
            if body:
                content_text = body.get_text(separator='\n', strip=True)

        # Làm sạch text
        # Loại bỏ whitespace thừa
        content_text = re.sub(r'\n\s*\n', '\n\n', content_text)
        # Loại bỏ các dòng quá ngắn (thường là menu, navigation)
        lines = content_text.split('\n')
        cleaned_lines = [line for line in lines if len(line.strip()) > 10]
        content_text = '\n'.join(cleaned_lines)

        return content_text

    except Exception as e:
        print(f"❌ Error crawling {url}: {e}")
        return ""

# Crawl văn bản
document_url = "https://www.moj.gov.vn/vbpq/lists/vn%20bn%20php%20lut/view_detail.aspx?itemid=23311"
print(f"📥 Crawling document from: {document_url}")
legal_text = crawl_legal_document(document_url)

print(f"\n✅ Crawled {len(legal_text):,} characters")
print(f"📄 Number of lines: {len(legal_text.splitlines())}")
print(f"\nPreview (first 1000 chars):\n{legal_text[:1000]}...")
print(f"\n..." + "="*60)
print(f"Preview (middle section):\n{legal_text[len(legal_text)//2:len(legal_text)//2+500]}...")

📥 Crawling document from: https://www.moj.gov.vn/vbpq/lists/vn%20bn%20php%20lut/view_detail.aspx?itemid=23311

✅ Crawled 62,877 characters
📄 Number of lines: 602

Preview (first 1000 chars):
Turn on more accessible mode
Turn off more accessible mode
Cổng thông tin điện tử Bộ Tư pháp - Ministry of Justice’s portal
Cổng thông tin điện tử
CSDLQG về văn bản pháp luật
Chương VIII
Cục Công nghệ thông tin, Bộ Tư pháp trân trọng cảm ơn Quý độc giả trong thời gian qua đã sử dụng hệ thống văn bản quy phạm pháp luật tại địa chỉ
http://www.moj.gov.vn/pages/vbpq.aspx
Đến nay, nhằm phục vụ tốt hơn nhu cầu khai thác, tra cứu văn bản quy phạm pháp luật từ Trung ương đến địa phương, Cục Công nghệ thông tin đã đưa Cơ sở dữ liệu quốc gia về văn bản pháp luật vào sử dụng tại địa chỉ
http://vbpl.vn/Pages/portal.aspx
để thay thế cho hệ thống cũ nói trên.
Cục Công nghệ thông tin trân trọng thông báo tới Quý độc giả được biết và mong rằng Cơ sở dữ liệu quốc gia về văn bản pháp luật sẽ tiếp tục là địa chỉ tin 

## 5. Chunking và tạo Documents

In [8]:
# Tạo Document từ text crawled
document = Document(text=legal_text)

# Chunking với SentenceSplitter
splitter = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=50,
    separator="\n"
)

nodes = splitter.get_nodes_from_documents([document])

print(f"✅ Created {len(nodes)} chunks")
print(f"\nSample chunk:")
print(f"{nodes[0].text[:300]}...")

✅ Created 69 chunks

Sample chunk:
Turn on more accessible mode
Turn off more accessible mode
Cổng thông tin điện tử Bộ Tư pháp - Ministry of Justice’s portal
Cổng thông tin điện tử
CSDLQG về văn bản pháp luật
Chương VIII
Cục Công nghệ thông tin, Bộ Tư pháp trân trọng cảm ơn Quý độc giả trong thời gian qua đã sử dụng hệ thống văn bản...


## 6. Tạo FAISS Vector Store và merge similar chunks

In [9]:
# Tạo FAISS index
dimension = 1536  # text-embedding-3-small dimension
faiss_index = faiss.IndexFlatL2(dimension)

# Tạo vector store
vector_store = FaissVectorStore(faiss_index=faiss_index)

# Tạo index từ nodes
print("🔄 Creating vector index...")
index = VectorStoreIndex(
    nodes=nodes,
    vector_store=vector_store,
    show_progress=True
)

print(f"✅ Vector index created with {len(nodes)} nodes")

🔄 Creating vector index...


Generating embeddings:   0%|          | 0/69 [00:00<?, ?it/s]

✅ Vector index created with 69 nodes


In [10]:
def merge_similar_chunks(nodes: List, similarity_threshold: float = 0.75) -> List[Dict]:
    """
    Merge các chunks có cosine similarity cao để tạo contexts phong phú hơn
    Với 182 chunks, threshold 0.75 sẽ tạo ra khoảng 60-90 contexts
    """
    print(f"🔄 Merging similar chunks (threshold={similarity_threshold})...")
    print(f"📊 Input: {len(nodes)} chunks")

    # Get embeddings cho tất cả nodes
    embeddings = []
    for node in tqdm(nodes, desc="Getting embeddings"):
        emb = embed_model.get_text_embedding(node.text)
        embeddings.append(emb)

    embeddings = np.array(embeddings)

    # Tính cosine similarity matrix
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_matrix = cosine_similarity(embeddings)

    # Merge chunks
    merged_contexts = []
    used_indices = set()

    for i in range(len(nodes)):
        if i in used_indices:
            continue

        # Tìm các chunks similar
        similar_indices = np.where(similarity_matrix[i] > similarity_threshold)[0]
        similar_indices = [idx for idx in similar_indices if idx not in used_indices]

        # Merge text
        merged_text = "\n\n".join([nodes[idx].text for idx in similar_indices])

        merged_contexts.append({
            'context': merged_text,
            'num_chunks': len(similar_indices),
            'chunk_indices': similar_indices  # Đã là list rồi, không cần .tolist()
        })

        used_indices.update(similar_indices)

    print(f"✅ Merged {len(nodes)} chunks into {len(merged_contexts)} contexts")
    print(f"📈 Avg chunks per context: {len(nodes)/len(merged_contexts):.1f}")
    return merged_contexts

# Merge chunks với threshold thấp hơn để tạo nhiều contexts hơn
merged_contexts = merge_similar_chunks(nodes, similarity_threshold=0.75)

🔄 Merging similar chunks (threshold=0.75)...
📊 Input: 69 chunks


Getting embeddings: 100%|██████████| 69/69 [00:27<00:00,  2.51it/s]


✅ Merged 69 chunks into 43 contexts
📈 Avg chunks per context: 1.6


In [11]:
len(merged_contexts)

43

## 7. Generate Questions từ Contexts

In [12]:
def generate_questions_from_context(context: str, num_questions: int = 6) -> List[str]:
    """
    Generate câu hỏi từ context sử dụng GPT-4o-mini
    """
    prompt = f"""Bạn là chuyên gia pháp luật Việt Nam. Dựa trên đoạn văn bản pháp luật dưới đây, hãy tạo {num_questions} câu hỏi hay và đa dạng mà người dùng có thể hỏi.

Văn bản pháp luật:
{context}

Yêu cầu:
- Câu hỏi phải tự nhiên, giống như người dùng thật sẽ hỏi
- Đa dạng về loại câu hỏi: hỏi định nghĩa, điều kiện, thủ tục, quyền lợi, trách nhiệm, v.v.
- Câu hỏi phải có thể trả lời được từ context trên
- Trả về ONLY danh sách các câu hỏi, mỗi câu một dòng, không đánh số

Câu hỏi:"""

    try:
        response = llm.complete(prompt)
        questions_text = response.text.strip()

        # Parse questions
        questions = [q.strip() for q in questions_text.split('\n') if q.strip()]
        # Loại bỏ numbering nếu có
        questions = [re.sub(r'^\d+\.\s*', '', q) for q in questions]

        return questions[:num_questions]
    except Exception as e:
        print(f"❌ Error generating questions: {e}")
        return []

# Test với 1 context
test_context = merged_contexts[0]['context']
test_questions = generate_questions_from_context(test_context, num_questions=6)

print("📝 Sample generated questions:")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")

📝 Sample generated questions:
1. Cổng thông tin điện tử của Bộ Tư pháp có chức năng gì?
2. Tại sao Cục Công nghệ thông tin lại quyết định thay thế hệ thống cũ bằng Cơ sở dữ liệu quốc gia về văn bản pháp luật?
3. Làm thế nào để tôi có thể tra cứu văn bản quy phạm pháp luật từ Trung ương đến địa phương?
4. Cơ sở dữ liệu quốc gia về văn bản pháp luật có gì khác biệt so với hệ thống cũ?
5. Tôi có thể gửi ý kiến góp ý về Cơ sở dữ liệu quốc gia ở đâu?
6. Có thể sử dụng Cơ sở dữ liệu quốc gia về văn bản pháp luật vào những mục đích nào?


## 8. Generate Answers từ Questions + Context

In [13]:
def generate_answer(question: str, context: str) -> str:
    """
    Generate câu trả lời từ question và context sử dụng GPT-4o-mini
    """
    prompt = f"""Bạn là trợ lý tư vấn pháp luật Việt Nam chuyên nghiệp. Hãy trả lời câu hỏi dưới đây dựa trên văn bản pháp luật được cung cấp.

Văn bản pháp luật:
{context}

Câu hỏi: {question}

Yêu cầu:
- Trả lời chính xác, đầy đủ dựa trên văn bản pháp luật
- Trích dẫn điều luật cụ thể nếu có
- Giải thích rõ ràng, dễ hiểu
- Nếu có nhiều trường hợp, liệt kê đầy đủ
- Độ dài: 50-200 từ

Câu trả lời:"""

    try:
        response = llm.complete(prompt)
        answer = response.text.strip()
        return answer
    except Exception as e:
        print(f"❌ Error generating answer: {e}")
        return ""

# Test với 1 question
test_answer = generate_answer(test_questions[0], test_context)

print("💬 Sample Q&A:")
print(f"Q: {test_questions[0]}")
print(f"A: {test_answer}")

💬 Sample Q&A:
Q: Cổng thông tin điện tử của Bộ Tư pháp có chức năng gì?
A: Cổng thông tin điện tử của Bộ Tư pháp có chức năng cung cấp và tra cứu văn bản quy phạm pháp luật từ Trung ương đến địa phương. Theo thông tin trong văn bản, Cục Công nghệ thông tin đã phát triển Cơ sở dữ liệu quốc gia về văn bản pháp luật nhằm phục vụ nhu cầu khai thác, tra cứu của người dân và các tổ chức. 

Cụ thể, cổng thông tin này cho phép người dùng tìm kiếm các văn bản pháp luật hiện hành, giúp họ tiếp cận thông tin pháp lý một cách nhanh chóng và thuận lợi. Đồng thời, cổng còn khuyến khích độc giả gửi ý kiến góp ý để cải thiện chất lượng và tính năng của hệ thống này. 

Như vậy, cổng thông tin không chỉ là nơi cung cấp thông tin mà còn là cầu nối giữa cơ quan nhà nước và người dân trong việc tiếp cận thông tin pháp luật.


## 9. Generate toàn bộ dataset (Target: 1000 Q&A pairs)

In [15]:
def generate_synthetic_dataset(
    contexts: List[Dict],
    target_samples: int = 860,
    questions_per_context: int = 20
) -> List[Dict]:
    """
    Generate toàn bộ synthetic dataset
    """
    dataset = []

    # Tính số contexts cần dùng
    num_contexts_needed = min(len(contexts), (target_samples // questions_per_context) + 1)

    print(f"🚀 Generating {target_samples} Q&A pairs from {num_contexts_needed} contexts...")

    for ctx_data in tqdm(contexts[:num_contexts_needed], desc="Processing contexts"):
        context = ctx_data['context']

        # Skip contexts quá ngắn
        if len(context.split()) < 50:
            continue

        # Generate questions
        questions = generate_questions_from_context(context, num_questions=questions_per_context)

        # Generate answers cho mỗi question
        for question in questions:
            if len(dataset) >= target_samples:
                break

            answer = generate_answer(question, context)

            if answer:  # Chỉ thêm nếu có answer
                dataset.append({
                    'question': question,
                    'answer': answer
                })

        if len(dataset) >= target_samples:
            break

    print(f"\n✅ Generated {len(dataset)} Q&A pairs")
    return dataset

# Generate dataset
synthetic_data = generate_synthetic_dataset(
    merged_contexts,
    target_samples=860,
    questions_per_context=20
)

🚀 Generating 860 Q&A pairs from 43 contexts...


Processing contexts:  98%|█████████▊| 42/43 [1:09:51<01:39, 99.81s/it]


✅ Generated 860 Q&A pairs


## 10. Lưu dataset dưới dạng JSONL

In [16]:
# Lưu ra file JSONL
output_path = 'synthetic_legal_qa_data.jsonl'

with open(output_path, 'w', encoding='utf-8') as f:
    for item in synthetic_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ Saved {len(synthetic_data)} Q&A pairs to {output_path}")

# Download file (nếu chạy trên Colab)
from google.colab import files
files.download(output_path)

✅ Saved 860 Q&A pairs to synthetic_legal_qa_data.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 11. Thống kê & Preview dataset

In [ ]:
import pandas as pd

# Tạo DataFrame để phân tích
df = pd.DataFrame(synthetic_data)

# Thống kê độ dài
df['question_len'] = df['question'].apply(lambda x: len(x.split()))
df['answer_len'] = df['answer'].apply(lambda x: len(x.split()))

print("📊 DATASET STATISTICS:")
print("=" * 60)
print(f"Total samples: {len(df):,}")
print(f"\nQuestion length (words):")
print(df['question_len'].describe())
print(f"\nAnswer length (words):")
print(df['answer_len'].describe())

# Preview samples
print("\n" + "=" * 60)
print("SAMPLE Q&A PAIRS:")
print("=" * 60)
for i in range(min(5, len(df))):
    print(f"\n[Sample {i+1}]")
    print(f"Q: {df.iloc[i]['question']}")
    print(f"A: {df.iloc[i]['answer'][:200]}...")
    print("-" * 60)

## 12. Validation & Quality Check

In [ ]:
# Kiểm tra chất lượng
print("🔍 QUALITY CHECKS:")
print("=" * 60)

# Check duplicates
duplicate_questions = df[df.duplicated(subset=['question'], keep=False)]
print(f"Duplicate questions: {len(duplicate_questions)}")

# Check empty
empty_answers = df[df['answer'].str.strip() == '']
print(f"Empty answers: {len(empty_answers)}")

# Check very short answers (< 10 words)
short_answers = df[df['answer_len'] < 10]
print(f"Very short answers (< 10 words): {len(short_answers)}")

# Check very long answers (> 300 words)
long_answers = df[df['answer_len'] > 300]
print(f"Very long answers (> 300 words): {len(long_answers)}")

print("\n✅ Quality check completed!")

🔍 QUALITY CHECKS:
Duplicate questions: 0
Empty answers: 0
Very short answers (< 10 words): 0
Very long answers (> 300 words): 1

✅ Quality check completed!
